# **Tutorial for DICGrid model component**

DIC Grid serves for reading the data available as a sequence of files with displacement measurements stored in a directory, accompanied with the measured load deflection curve. It refers to the `BeamDesign` object defining the design of the reinforced beam. By combining the information from the optical measurement with statical scheme, geometry and reinforcement layout, this model component serves 
further analysis by providing the correct **load-space-time** databasis.

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pylab as plt
import collections
collections.Iterable = collections.abc.Iterable
from scipy.spatial import Delaunay

Let us construct a grid example and render the interactive interface showing the measured displacmeent at the maximum load $t = 1$ and with displacement amplified by a factor $U_\mathrm{factor} = 100$. 

In [ ]:
from bmcs_shear.api import DICGridTri
dic_grid = DICGridTri(U_factor=100, dir_name='B10_TV1', t=1, padding=40, d_x=3, d_y=3, n_T=40)
dic_grid.read_beam_design()

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
dic_grid.plot_grid_on_triangulation(ax)

In [ ]:
dic_grid.interact()

Note that the orange markers denote the states with available DIC measurements performed during the history of loading.

## Read data format

### Input directory structure

The `DICGrid` objects reads the data from a directory named as `dir_name`

In [ ]:
dic_grid.dir_name

The property trait `data_dir` assembles the path to the data directory using path starting in the home directory, i.e. 

```join(dic_grid.home_dir, 'simdb', 'data', 'shear_zone', dic_grid.dir_name)```

For the current example, we obtain

In [ ]:
dic_grid.data_dir

To get the directory with the DIC data, issue 

In [ ]:
dic_grid.dic_data_dir

### Load deflection response

To obtain the directory with the load deflection curve, use

In [ ]:
dic_grid.Fw_data_dir

### Beam parameters
File specifying the beam parameters is assessed using the property traits `beam_param_fie` and is named `beam_params.txt` by default

In [ ]:
dic_grid.beam_param_file

This file specifies the parameters 
 - length $L$
 - width $B$
 - depth $H$
 - number of reinforcement bars $n_\mathrm{s}$
 - vertical position of the bars $y_\mathrm{s}$
 - diameter of the reinforcement $d_\mathrm{s}$

In [ ]:
dic_grid.beam_param_types

Currently, to setup the beam parmaters correctly, a call to `read_beam_design` must be issued using

In [ ]:
dic_grid.read_beam_design()

**Note** This need should be avoided later by capturing the dependency with respect to the `data_dir` attribute. Currently, the notification mechanism does not allow this because of are further dependency links.

The beam design object is available in the attribute `sz_bd`. The type of the `RCBeamDesign` is general and specifies also the material models associated with individual material components.

The parameters relevant for correct configuration of the `dic_grid` within the beam design are the length and offset with respect the beam bounaries 

In [ ]:
dic_grid.asc_F_files

In [ ]:
dic_grid.L_x, dic_grid.L_y

### Grid parameters

File specifying the grid data parameters is accessed using the property trait `grid_param_file` and is named
`grid_params.txt` by default.

In [ ]:
dic_grid.grid_param_file

This file contains the name-value pairs of the attributes defined by the `grid_param_types` dictionary

In [ ]:
dic_grid.grid_param_types

Thus, the `grid_params` of the current test `B7_TV1` read

In [ ]:
dic_grid.grid_params

The DIC data files corresponding to each measured step are assembled in the string list `files` 

These parameters are available as properties and can be accessed via

In [ ]:
dic_grid.n_I, dic_grid.n_J, dic_grid.d_x, dic_grid.d_y, dic_grid.x_offset, dic_grid.y_offset

Note that the naming of properties follows the mathematical indexing convention with $I$, $J$ representing the horizontal and vertical node indexes, respectively. Correspondingly, $n_I$ and $n_J$ denotes the number of horizontal and vertical nodes, respectively. 

To position the DIC frame into the beam, the beam parameters and grid parameters are combined to obtain the coordinates of the 
bottom left and top right corners within the beam in the order $X_\mathrm{frame} = (x_\min, y_\min, x_\max, y_\max)$

In [ ]:
dic_grid.X_frame

In [ ]:
dic_grid.asc_F_files

Note that the `CSV` files are named according to the scheme specifying their load levels

## Time data

The load `F` corresponding to each history index `T` is provided in the `F_dic_T` property.

In [ ]:
dic_grid.F_dic_T

The index corresponding to the maximum load `F` is accessed via

The final index of snapshots is also available as `n_T`

In [ ]:
dic_grid.n_T

An instance of `dic_grid` keeps track of the current time $t \in (0, 1)$ with 0 
and 1 representing the states with zero and maximum force, respectively. Thus, by setting

In [ ]:
dic_grid.t = 0.5

The grid will return displacement and position values corresponding to the half of the maximum load. When setting the value of $t$, the value of `T1` denoting the history index will be adapted automateically, i.e.

In [ ]:
dic_grid.T0, dic_grid.n_T

The whole load and deflection values for each history index $T$ are available in the table `Fw_T`, representing the raw format of the measured data. The 

In [ ]:
dic_grid.Fw_T.shape

To access the deflections and forces, we need to issue

In [ ]:
w = dic_grid.Fw_T[::50,2]
F = -dic_grid.Fw_T[::50,1]

**TODO**: provide properties

In [ ]:
dic_grid.t = 1
dic_grid.F_T_t

## Grid data

The initial positions of the grid markers denoted as $X_{0IJa}$ is obtained from the `grid_params`. The initial position of the 
bottom left corner must match the values of the offset specified above in the $X_\mathrm{frame}$. Let us verify that this feature is provided by printing the values of `X_0IJa[0,0,:]` and `X_0IJa[-1,-1,:]`.

In [ ]:
dic_grid.X_frame, dic_grid.X_IJa[0, 0, :], dic_grid.X_IJa[-1, -1, :]

All files are read into the memory at once. This might be improvied in the future using the zipped binary `numpy` file management to save memory during processing.

The displacements of the grid nodes are stored in a four-dimensional array 
$U_{TIJa}$ with the indexes denoting the history index $T$, horizontal 
node index $I$, vertical index $J$, and the dimension index $a \in (0,1) = (x,y)$.
Thus, the displacement at the fifteenth step of the left bottom corner node
in both directions can be obtained as

In [ ]:
dic_grid.U_TIJa[-1, 40, 0, :]

The displacement corresponding to the current value of $t$ represented by the index T_t

In [ ]:
dic_grid.t, dic_grid.T_t

is available as property $U_{IJa}$

In [ ]:
dic_grid.U_IJa[40, 0, :]

If a new time is set

In [ ]:
dic_grid.t = 0.6

A new value of displacement is obtained as

In [ ]:
dic_grid.U_IJa[40, 0, :]

# Plotting functions

In [ ]:
%matplotlib widget
_, ax = plt.subplots(1,1)
dic_grid.t = 0.2
dic_grid.plot_load_deflection(ax)

**Remark**: Correct the scaling - t should be between the min max values of the load

In [ ]:
_, ax = plt.subplots(1,1)
dic_grid.plot_grid(ax)

In [ ]:
_, ax = plt.subplots(1,1)
dic_grid.plot_bounding_box(ax)
dic_grid.plot_box_annotate(ax)
ax.set_aspect('equal')

# Use case - loop over time steps

In [ ]:
import numpy as np
dic_grid.t_dic_T

In [ ]:
dic_grid.t_dic_T[::-1]

In [ ]:
T_dic_t = []
for t_dic in dic_grid.t_dic_T:
    dic_grid.t = t_dic
    T_dic_t.append(dic_grid.T_t)
T_dic_t = np.array(T_dic_t)

In [ ]:
dic_grid.U_TIJa[T_dic_t,0,-1,1] - dic_grid.U_TIJa[:,0,-1,1] 

Interpolation along the time domain is possible using the time index $T$, true time $t$, deflection $w$ or the force $F$.
Time index $T$ refers to the spatial snapshots of dic. 

The load deflection curve is provided in the form $F(t), w(t)$, where $t \in \mathcal{T}$ is the true time. 
The snapshots with DIC are obtained at times $t_\mathrm{dic} \in \mathcal{T}_\mathrm{dic} \subset \mathcal{T}$.

In [ ]:
import numpy as np
t, F, w = dic_grid.Fw_T[:,(0,1,2)].T
argmax_F = np.argmax(-F)
-F[:argmax_F]

In [ ]:
t, F, w = [ timeline[:argmax_F] for timeline in [t, -F, w] ] 

In [ ]:
F

In [ ]:
dic_grid.F_dic_T

In [ ]:
dic_grid.U_TIJa[:,0,-1,0].shape, dic_grid.F_dic_T.shape

In [ ]:
idx = 0
idx_list = []
for F_dic in dic_grid.F_dic_T:
    idx += np.argmax(-F[idx:] >= F_dic)
    idx_list.append(idx)
idx_ = np.array(idx_list)

In [ ]:
-F[idx_]

In [ ]:
t = 0.96
self = dic_grid
F = -self.Fw_T[::50,1]
F_max = np.max(F)
F_t = t * F_max
F_dic_T = self.F_dic_T
dic_T = np.arange(len(F_dic_T))
T_t = np.interp(F_t, F_dic_T, dic_T)
int(T_t)

In [ ]:
dic_T